# 不等式のQUBO変換を用いて基底エネルギー以下のサンプルを得る
- small data
- dwave-neal
- 基底エネルギーは0であるべき


### 量子アニーリングって
SAの量子版




### import

In [44]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import random
from pyqubo import Array, OneHotEncInteger, solve_qubo
import scipy.stats

### 各変数

In [45]:
# X
#matrix = np.random.randn(5,5)
#X_ori= pd.DataFrame(matrix, columns=list('ABCDE'))
#X_ori

# CSV ファイル (df1.csv) として出力
#X_ori.to_csv("../input/df1.csv")

In [47]:
X_ori = pd.read_csv("../input/df1.csv", sep=',', index_col=0)
X_ori

,A,B,C,D,E
0,-1.297303,-0.049925,0.185726,-0.580797,0.926729
1,1.129216,0.263337,-2.072408,0.269691,-0.565672
2,-1.140225,0.447902,-0.641875,-0.032109,0.415254
3,-1.004241,0.700738,-1.250194,-1.827180,1.972915
4,1.119464,0.248735,1.328148,0.295280,1.007603


In [48]:
# y
ori_y = pd.Series([1, 0, 1, 1, 0])
ori_y

0    1
1    0
2    1
3    1
4    0
dtype: int64

In [49]:
selected_col_num = 2

## 特徴量選択
 - select_cols, else_colsを決める

In [50]:
selector = SelectKBest(score_func=f_regression, k=selected_col_num) 
selector.fit(X_ori, ori_y)
mask = selector.get_support() 

In [51]:
select_cols = []
else_cols = []
for ii in range(len(mask)):
    if mask[ii] == 0:
        else_cols.append(X_ori.columns[ii])
    else:
         select_cols.append(X_ori.columns[ii])

select_cols

['A', 'D']

## すべての不等式が成り立つ：条件
したいこと→ハミルトニアンを最小化させる（xを最適化）

In [52]:
from decimal import *

In [54]:
#Xの要素を標準化(Standardization : 平均0分散1)
X_standardization = pd.DataFrame(scipy.stats.zscore(X_ori), columns=X_ori.columns)
X_standardization

,A,B,C,D,E
0,-0.948036,-1.502995,0.576689,-0.259926,0.211537
1,1.224875,-0.237599,-1.350141,0.814379,-1.588719
2,-0.807376,0.507934,-0.129490,0.433156,-0.405446
3,-0.685604,1.529245,-0.648559,-1.834311,1.473533
4,1.216142,-0.296585,1.551500,0.846703,0.309095


In [56]:
max_X_element = X.max(axis=1).max() 
min_X_element = X.min(axis=1).min() 

num_samples = X.shape[0]
num_cols = X.shape[1]

In [57]:
# W : ay - byの最大値

#xの要素で最大値とxの要素の最小値を知れば推定はできる
W = int((max_X_element *1*num_samples - min_X_element*1*num_samples)//1)

In [58]:
def make_y(y_before):
    select_index = random.randrange(num_samples)
    y_before[select_index] = 1 - y_before[select_index] 
    y_after = y_before
    return y_after

In [59]:
def make_ax_by(y_after):
    ax_by = []
    for i in range(selected_col_num):
        for j in range(num_cols - selected_col_num):
            ax_by.append(np.dot(X[select_cols[i]], y_after) - np.dot(X[else_cols[j]], y_after) +1)
    return ax_by 

In [60]:
def H(ax_by, index_ax_by):
    z = [0]*W
        
    each_ax_by = ax_by[index_ax_by]
    int_each_axby = each_ax_by //1
    sum_w_z = 0
    
    if int_each_axby >= 1:
        z[int(int_each_axby)] = 1
        sum_w_z = sum([int_each_axby*z[w] for w in range(W)])
    else:
        z[0] = 1
        sum_w_z = sum([1*z[w] for w in range(W)])
        
    sum_z = sum(z)

    ans_h =  (1-sum_z)**2 + (sum_w_z -  int(int_each_axby))**2
    return ans_h

In [61]:
def H_sum(ax_by):
        H_sum =  sum([H(ax_by, index_ax_by) for index_ax_by in range(len(ax_by))])
        return H_sum

In [62]:
ax_by = make_ax_by(ori_y)
ori_H_sum = H_sum(ax_by)
ori_H_sum

ValueError: shapes (3,) and (5,) not aligned: 3 (dim 0) != 5 (dim 0)

## neal.sampler.SimulatedAnnealingSampler.sample_qubo

**SimulatedAnnealingSampler.sample_qubo(Q, **parameters)**

Sample from a QUBO using the implemented sample method.

This method is inherited from the Sampler base class.

Converts the QUBO into a BinaryQuadraticModel and then calls sample().

Parameters:	
- Q (dict) – Coefficients of a quadratic unconstrained binary optimization (QUBO) problem. Should be a dict of the form {(u, v): bias, …} where u, v, are binary-valued variables and bias is their associated coefficient.
- **kwargs – See the implemented sampling for additional keyword definitions.

Returns:	
- SampleSet

In [11]:
y = Array.create('y', shape=num_samples, vartype='BINARY')

In [12]:
m_list = []
for i in range(selected_col_num):
    for j in range(num_cols - selected_col_num):
        sel_list  = X[select_cols[i]].values.tolist()
        else_list  = X[else_cols[j]].values.tolist()
        m_list.append(y.dot(Array(sel_list)) - y.dot(Array(else_list)))

In [13]:
m_list

[((Binary(y[0])*Num(0))+(Binary(y[1])*Num(2))+(Binary(y[2])*Num(-2))+(((Binary(y[0])*Num(1))+(Binary(y[1])*Num(0))+(Binary(y[2])*Num(3)))*Num(-1))),
 ((Binary(y[0])*Num(0))+(Binary(y[1])*Num(2))+(Binary(y[2])*Num(-2))+(((Binary(y[0])*Num(-1))+(Binary(y[1])*Num(0))+(Binary(y[2])*Num(0)))*Num(-1))),
 ((Binary(y[0])*Num(0))+(Binary(y[1])*Num(2))+(Binary(y[2])*Num(-2))+(((Binary(y[0])*Num(0))+(Binary(y[1])*Num(0))+(Binary(y[2])*Num(0)))*Num(-1))),
 ((Binary(y[0])*Num(0))+(Binary(y[1])*Num(2))+(Binary(y[2])*Num(-2))+(((Binary(y[0])*Num(2))+(Binary(y[1])*Num(2))+(Binary(y[2])*Num(-1)))*Num(-1)))]

In [14]:
OneHotEncInteger("x", 1, W, strength = 1)

WithPenalty(value=((Binary(x[0])*Num(0))+(Binary(x[1])*Num(1))+(Binary(x[2])*Num(2))+(Binary(x[3])*Num(3))+(Binary(x[4])*Num(4))+(Binary(x[5])*Num(5))+(Binary(x[6])*Num(6))+(Binary(x[7])*Num(7))+(Binary(x[8])*Num(8))+(Binary(x[9])*Num(9))+(Binary(x[10])*Num(10))+(Binary(x[11])*Num(11))+(Binary(x[12])*Num(12))+(Binary(x[13])*Num(13))+(Binary(x[14])*Num(14))+Num(1)),penalty=(Const(x_const, ((Binary(x[0])+Binary(x[1])+Binary(x[2])+Binary(x[3])+Binary(x[4])+Binary(x[5])+Binary(x[6])+Binary(x[7])+Binary(x[8])+Binary(x[9])+Binary(x[10])+Binary(x[11])+Binary(x[12])+Binary(x[13])+Binary(x[14])+Num(-1))*(Binary(x[0])+Binary(x[1])+Binary(x[2])+Binary(x[3])+Binary(x[4])+Binary(x[5])+Binary(x[6])+Binary(x[7])+Binary(x[8])+Binary(x[9])+Binary(x[10])+Binary(x[11])+Binary(x[12])+Binary(x[13])+Binary(x[14])+Num(-1))))*Num(1)))

WithPenalty(value=((Binary(x[0])*Num(0))+(Binary(x[1])*Num(1))+(Binary(x[2])*Num(2))+(Binary(x[3])*Num(3))+(Binary(x[4])*Num(4))+(Binary(x[5])*Num(5))+(Binary(x[6])*Num(6))+(Binary(x[7])*Num(7))+(Binary(x[8])*Num(8))+(Binary(x[9])*Num(9))+(Binary(x[10])*Num(10))+(Binary(x[11])*Num(11))+Num(1)),penalty=(Const(x_const, ((Binary(x[0])+Binary(x[1])+Binary(x[2])+Binary(x[3])+Binary(x[4])+Binary(x[5])+Binary(x[6])+Binary(x[7])+Binary(x[8])+Binary(x[9])+Binary(x[10])+Binary(x[11])+Num(-1))*(Binary(x[0])+Binary(x[1])+Binary(x[2])+Binary(x[3])+Binary(x[4])+Binary(x[5])+Binary(x[6])+Binary(x[7])+Binary(x[8])+Binary(x[9])+Binary(x[10])+Binary(x[11])+Num(-1))))*Num(1)))

In [15]:
H_sum = sum((OneHotEncInteger("x", 1, W, strength = 1)-m)**2 for m in m_list)
model = H_sum.compile()

In [16]:
qubo, offset = model.to_qubo()

#PyQUBOによるSA
raw_solution = solve_qubo(qubo)

# 得られた結果をデコードする
decoded_solution, broken, energy = model.decode_solution(raw_solution, vartype="BINARY")
decoded_solution["y"] 

{0: 0, 1: 1, 2: 0}

In [17]:
decoded_solution

{'x': {0: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0,
  1: 0,
  2: 1,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0},
 'y': {0: 0, 1: 1, 2: 0}}

In [18]:
broken

{}

In [19]:
energy 

12.0

#### 課題
すべての$x$を出力したいが、できていない

<br/>

### 元のyで求められるエネルギーとenergy を比較したい
- ori_H_sum : 元のyで求められるエネルギー

In [20]:
def make_ax_by(y):
    ax_by = []
    for i in range(selected_col_num):
        for j in range(num_cols - selected_col_num):
            ax_by.append(np.dot(X[select_cols[i]], y) - np.dot(X[else_cols[j]], y))
    return ax_by 

In [21]:
def H(ax_by, index_ax_by):
    z = [0]*W
        
    each_axby = ax_by[index_ax_by]
    sum_w_z = 0
    
    if each_axby > 0:
        z[int(each_axby)] = 1
        sum_w_z = sum([each_axby*z[w] for w in range(W)]) #正確には間違っているが、zの要素は1つ1がある以外全て0なので計算結果は合う
    else:
        z[0] = 1
        sum_w_z = sum([1*z[w] for w in range(W)]) #一番小さいHを得られるのは z[0] = 1であるため、こう設定した
        
    sum_z = sum(z)
    z_series = pd.Series(z)

    ans_h =  (1-sum_z)**2 + (sum_w_z -  int(each_axby))**2
    return ans_h,  z_series

In [22]:
#know ori_H_sum
ax_by = make_ax_by(ori_y)

ori_H_sum =  sum([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
ori_z_list =  [H(ax_by, index_ax_by)[1] for index_ax_by in range(len(ax_by))]
ori_z_df = pd.concat(ori_z_list, axis = 1)

print(ori_H_sum)
print([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
print(ori_z_df)

78
[49, 4, 9, 16]
    0  1  2  3
0   1  1  1  1
1   0  0  0  0
2   0  0  0  0
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
11  0  0  0  0
12  0  0  0  0
13  0  0  0  0
14  0  0  0  0


In [23]:
if ori_H_sum >= energy:
    print('SAは成功していると考えられる')

SAは成功していると考えられる


## SAで出したyが全ての不等式を満たしているかどうか

In [24]:
sol_y =  pd.Series([0, 1, 0])

In [25]:
sol_ax_by = make_ax_by(sol_y)
sol_ax_by 

[2, 2, 2, 0]

In [26]:
ok = 0
for  ii in sol_ax_by :
    if ii >= 0:
        ok += 1
if ok == len(sol_ax_by):
    print('SAで出したyは全ての不等式を満たしている')

SAで出したyは全ての不等式を満たしている


## 結果
- `SAは成功していると考えられる`, `SAで出したyは全ての不等式を満たしている`両方を満たしていない結果も出る